In [1]:
1*2

2

In [2]:
import os
import GPUtil
import importlib

def sgpu():
    GPUtil.showUtilization()

def rl(module):
    importlib.reload(module)

In [3]:
sgpu()

| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [4]:
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import torch
import gpytorch

from scipy.stats import pearsonr
from scipy.special import binom as binom
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [5]:
torch.__version__

'1.9.0a0+2ecb2c7'

In [6]:
import sys
sys.path.insert(1, '../')


import EpiK.models as models

In [7]:
output_device = 0
n_devices = torch.cuda.device_count()
models.set_params(output_device, n_devices)
print("number of GPUs = {}; output device = {}".
      format(n_devices, torch.cuda.current_device()))

number of GPUs = 8; output device = 0


In [8]:
torch.__version__

'1.9.0a0+2ecb2c7'

In [9]:
from EpiK.functions import get_data, get_envs, set_data_path
set_data_path("../matsui_data/")

In [10]:
# training sizes
props = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95]

In [39]:
# check_point sizes
partitions = [2, 2, 2, 2, 4, 4, 4, 4, 16, 32, 64, 120]
pd.DataFrame({"props":props, "partitions":partitions}).to_csv("partition_sizes.csv", index=None)

In [40]:
partition_sizes = pd.read_csv("partition_sizes.csv")

In [41]:
partition_sizes

,props,partitions
0,0.01,2
1,0.05,2
2,0.10,2
3,0.20,2
4,0.30,4
5,0.40,4
6,0.50,4
7,0.60,4
8,0.70,16
9,0.80,32


### Data

In [13]:
env_list = get_envs()
env = env_list[5]

In [14]:
geno_t, pheno = get_data(env)

../EpiK/functions.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  geno_t = torch.tensor(geno_t, dtype=torch.float)


In [15]:
inds_sub = np.where(np.array(pheno.pheno < -0.6) == False)[0]

### Get R2 curve

In [16]:
train_sizes = np.round(np.array(props)*len(inds_sub)).astype('int')

In [17]:
# results = pd.DataFrame({"prop":props, "train_size": train_sizes, "r2_add+dom":np.zeros(len(props)), 
#               "r2_pair_epi":np.zeros(len(props)), "r2_pair":np.zeros(len(props))})

# results.to_csv("results_low_order.csv", index=None)

In [18]:
results = pd.read_csv("results_low_order.csv")

In [19]:
model_names = ["add+dom", "pair_epi", "pair"]

In [20]:
from EpiK.kernels import K1, K2, K20
kernels = dict(zip(model_names, [K1, K20, K2]))

In [21]:
b = 5.
log_par = [0., 0., -b, -b, -b]

In [22]:
kernel_inits = dict(zip(model_names, [log_par[:2], log_par[1:3], log_par]))
kernel_inits

{'add+dom': [0.0, 0.0],
 'pair_epi': [0.0, -5.0],
 'pair': [0.0, 0.0, -5.0, -5.0, -5.0]}

In [31]:
from EpiK.functions import get_train_test, train_model_cv

np.random.seed(200)
train_size = 10000
sub = np.random.choice(inds_sub, train_size)
sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 4000)
train_x, test_x, train_y, test_y = get_train_test(geno_t, pheno, sub, sub_t)

In [32]:
kers_trained = {}
likelihoods_trained = {}

In [33]:
from EpiK.functions import train_model

In [34]:
checkpoint_size = 0
preconditioner_size = 100

for i in range(1, 3):
    model_name = model_names[i]
    ker = kernels[model_name]()
    log_raw_par = torch.tensor(kernel_inits[model_name])
    ker.raw_par = torch.nn.Parameter(log_raw_par)
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
    model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)
    train_model(model, 
            likelihood, 
            train_x, 
            train_y, 
            checkpoint_size, 
            preconditioner_size, 
            training_iter=150, 
            lr=.05)
    kers_trained[model_name] = model.covar_module.module
    likelihoods_trained[model_name] = likelihood

0
20
40
60
80
100
120
140
0
20
40
60
80
100
120
140


In [37]:
partition_size

0.05

In [42]:
likelihood = likelihoods_trained[model_name]
ker = kers_trained[model_name]

for i in range(1, len(props)):
    
    print("working on training data proportion = {}".format(props[i]))
    
    if results["r2_"+model_name][i] != 0.:
        print("r2_score found, skipping to next")
    
    else:
        print("no r2_score recorded, proceeding to calculate")
        np.random.seed(100)
        train_size = np.round(props[i]*len(inds_sub)).astype('int')
        sub = np.random.choice(inds_sub, train_size)
        sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 5000)
        train_x, test_x, train_y, test_y = get_train_test(geno_t, pheno, sub, sub_t)

        # make predictions - build model
        torch.cuda.empty_cache()

        model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)
        
        partition_size = partition_sizes.iloc[i, 1]

        
        model.eval()
        likelihood.eval()
        with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
            f_preds = model(test_x)

        f_mean = f_preds.mean.cpu().detach().numpy()
        y_test = test_y.detach().cpu().numpy()
        r2_score = r2(y_test, f_mean)                
        results["r2_"+model_name][i] = r2_score
        
#         # make predictions - loop to increase partition_size until passes
#         loop = True
#         while loop:
#             try: 
#                 partition_size = partition_sizes.iloc[i, 1]
#                 print("try doing inference under partition size = {}".format(partition_size))
#                 import gc
#                 gc.collect()
#                 torch.cuda.empty_cache()                

#                 model.eval()
#                 likelihood.eval()
#                 with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
#                     f_preds = model(test_x)
                    
#                 f_mean = f_preds.mean.cpu().detach().numpy()
#                 y_test = test_y.detach().cpu().numpy()
#                 r2_score = r2(y_test, f_mean)                
#                 results.iloc[i, 2] = r2_score
#                 loop = False            

#             except: 
#                 print("failed on current partition_size, increasing by 5")
#                 partition_sizes.iloc[i, 1] = partition_sizes.iloc[i,1] + 5
#                 partition_sizes.to_csv("partition_sizes.csv")            

working on training data proportion = 0.05
no r2_score recorded, proceeding to calculate


RuntimeError: The kernel MultiDeviceKernel is not equipped to handle and diag. Expected size torch.Size([9397]). Got size torch.Size([5, 9397])

In [35]:
model_name = model_names[0]

In [36]:
ker = kernels[model_name]()
log_raw_par = torch.tensor(kernel_inits[model_name])
ker.raw_par = torch.nn.Parameter(log_raw_par)

In [37]:
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

In [38]:
from EpiK.functions import train_model

In [39]:
checkpoint_size = 0
preconditioner_size = 100

train_model(model, 
            likelihood, 
            train_x, 
            train_y, 
            checkpoint_size, 
            preconditioner_size, 
            training_iter=200, 
            lr=.05)

0


/home/juannanzhou/.local/lib/python3.8/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2111211.25 which is larger than the tolerance of 1 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/juannanzhou/.local/lib/python3.8/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations with average residual norm 283547.0 which is larger than the tolerance of 1 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/juannanzhou/.local/lib/python3.8/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations wit

In [34]:
torch.__version__

'1.8.1'

In [21]:
import EpiK.functions
rl(EpiK.functions)

In [24]:
model_name = model_names[1]

In [25]:
from EpiK.functions import get_train_test, train_model_cv

for i in range(1, len(props)):
    
    print("working on training data proportion = {}".format(props[i]))
    
    if results["r2_"+model_name][i] != 0.:
        print("r2_score found, skipping to next")
    
    else:
        print("no r2_score recorded, proceeding to calculate")
        np.random.seed(100)
        train_size = np.round(props[i]*len(inds_sub)).astype('int')
        sub = np.random.choice(inds_sub, train_size)
        sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 4000)
        train_x, test_x, train_y, test_y = get_train_test(geno_t, pheno, sub, sub_t)
        ker = kernels[model_name]()
        log_raw_par = torch.tensor(kernel_inits[model_name])
        ker.raw_par = torch.nn.Parameter(log_raw_par)

        # train model
        print("training GP model using CV")
        ker, likelihood = train_model_cv(ker, train_x, train_y, 200, .05)
        

        # make predictions - build model
        torch.cuda.empty_cache()
        model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

        # make predictions - loop to increase partition_size until passes
        loop = True
        while loop:
            try: 
                partition_size = partition_sizes.iloc[i, 1]
                print("try doing inference under partition size = {}".format(partition_size))
                import gc
                gc.collect()
                torch.cuda.empty_cache()                

                model.eval()
                likelihood.eval()
                with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
                    f_preds = model(test_x)
                    
                f_mean = f_preds.mean.cpu().detach().numpy()
                y_test = test_y.detach().cpu().numpy()
                r2_score = r2(y_test, f_mean)                
                results.iloc[i, 2] = r2_score
                loop = False            

            except: 
                print("failed on current partition_size, increasing by 5")
                partition_sizes.iloc[i, 1] = partition_sizes.iloc[i,1] + 5
                partition_sizes.to_csv("partition_sizes.csv")            

working on training data proportion = 0.05
no r2_score recorded, proceeding to calculate
training GP model using CV
working on iteration 0


/home/juannanzhou/.local/lib/python3.9/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.07770030200481415 which is larger than the tolerance of 0.01 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/juannanzhou/.local/lib/python3.9/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.0997861698269844 which is larger than the tolerance of 0.01 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/juannanzhou/.local/lib/python3.9/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminat

KeyboardInterrupt: 

In [59]:
train_size = np.round(props[5]*len(inds_sub)).astype('int')
sub = np.random.choice(inds_sub, train_size)
sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 4000)
train_x, test_x, train_y, test_y = get_train_test(geno_t, pheno, sub, sub_t)
ker = kernels[model_name]()
log_raw_par = torch.tensor(kernel_inits[model_name])
ker.raw_par = torch.nn.Parameter(log_raw_par)


In [60]:
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

In [62]:
model.eval()

ExactGPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): MultiDeviceKernel(
    (module): K1(
      (raw_par_constraint): LessThan(0.000E+00)
    )
  )
)

In [64]:
train_x.shape

torch.Size([75176, 7742])

In [ ]:
def train_model(model, 
                likelihood, 
                train_x, 
                train_y, 
                checkpoint_size, 
                preconditioner_size, 
                training_iter=300, 
                lr=.05):


In [56]:
# train model
print("training GP model using CV")
ker, likelihood = train_model_cv(ker, train_x, train_y, 200, 1)


training GP model using CV
working on iteration 0


/home/juannanzhou/.local/lib/python3.9/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations with average residual norm 460583.0625 which is larger than the tolerance of 0.01 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  warnings.warn(


KeyboardInterrupt: 

In [82]:
tr_size = np.min([20000, round(.5*train_x.shape[0])])
val_size = np.min([10000, round(train_x.shape[0] - tr_size)])

sub_tr = np.random.choice(range(len(train_x)), tr_size)
sub_val = np.random.choice(list(set(range(len(train_x))).difference(sub_tr)), val_size)
tr_x = train_x[sub_tr]
tr_y = train_y[sub_tr]
val_x = train_x[sub_val]
val_y = train_y[sub_val]




In [84]:
from EpiK.models import ExactGPModel

In [85]:
"""fitting hyperparameters of model by maximizing marginal log likelihood"""
# Use the adam optimizer, this includes GaussianLikelihood parameters
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
model = ExactGPModel(tr_x, tr_y, likelihood, ker).to(output_device)    

In [88]:
optimizer = torch.optim.AdamW(model.parameters(), .05)

for i in range(10):
    if i%10 == 0:
        print("working on iteration %i"%i)
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    model.eval()
    f_preds = model(val_x).mean

    # Calc loss and backprop gradients
    loss = torch.norm(f_preds - val_y)
    model.train()
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    del loss
del model

return ker, likelihood


working on iteration 0


TypeError: num_outputs_per_input() missing 1 required positional argument: 'x2'

In [95]:
model.eval()

ExactGPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): MultiDeviceKernel()
)

In [96]:
val_x.shape

torch.Size([939, 7742])

In [99]:
model = ExactGPModel(tr_x, tr_y, likelihood, ker).to(output_device)    

In [100]:
model.eval()

ExactGPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): MultiDeviceKernel()
)

In [104]:
ker

EpiK.kernels.K1

In [118]:
kernels[model_name]()

KeyError: "attribute 'raw_par' already exists"

In [119]:
ker = K1()

KeyError: "attribute 'raw_par' already exists"

In [123]:
from EpiK.kernels import Kernel
from gpytorch.constraints import Positive
from gpytorch.constraints import LessThan


class K1(Kernel):
    """Add + Dom kernel"""

    is_stationary = True

    # We will register the parameter when initializing the kernel
    def __init__(self, 
                par_prior=None, par_constraint=None, 
                **kwargs):
      super().__init__(**kwargs)

      # register the raw parameter
      self.register_parameter(
          name='raw_par', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 2))
      )

      # set the parameter constraint to be positive, when nothing is specified
      if par_constraint is None:
          par_constraint = LessThan(upper_bound=0.)

      # register the constraint
      self.register_constraint("raw_par", par_constraint)


    # now set up the 'actual' paramter
    @property
    def par(self):
      # when accessing the parameter, apply the constraint transform
      return self.raw_par_constraint.transform(self.raw_par)
    @par.setter
    def par(self, value):
      return self._set_par(value)



    def forward(self, geno1, geno2, **params):

        global L
        L = geno1.shape[1]
        
        geno1_ht = 1.*(geno1 == 1.)
        geno2_ht = 1.*(geno2 == 1.)        
        geno1_h0 = 1.*(geno1 == 0.)
        geno1_h1 = 1.*(geno1 == 2.)
        geno2_h0 = 1.*(geno2 == 0.)
        geno2_h1 = 1.*(geno2 == 2.)

        S1 = self.covar_dist(geno1_ht, geno2_ht, **params)
        S2 = self.covar_dist(geno1_h0, geno2_h0, **params) + self.covar_dist(geno1_h1, geno2_h1, **params)
        D2 = self.covar_dist(geno1_h0, geno2_h1, **params) + self.covar_dist(geno1_h1, geno2_h0, **params)
        D1 = L - S1 - S2 - D2

        return torch.exp(self.par[0])*k_1_0(S1, S2, D1, D2) + torch.exp(self.par[1])*k_0_1(S1, S2, D1, D2)


In [125]:
ker = K1()